In [ ]:
import os, sys
import scipy.misc
from glob import glob
import numpy as np
import random 
import shutil 
import cv2
import matplotlib.pyplot as plt
from keras.models import load_model
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization


base_path = os.path.abspath("../")

sys.path.append(base_path)
import helpers

In [ ]:
def predict(model, img):
    
    if img.shape[0] != 256:
        img = scipy.misc.imresize(img, (256,256))
    
    # Normalize image as the trained distribution
    img = img/127.5 - 1.
    
    img = np.expand_dims(img, axis=0)
    img = model.predict(img)
    img = np.squeeze(img, axis=0)
    
    # Rescale to [0,1]
    img = 0.5 * img + 0.5 
    img = img * 255.0
    return img


def load_img(path):
    # read image
    img = scipy.misc.imread(path, mode='RGB').astype(np.float)
    # predict
    img = predict(model, img)
    return img 

In [ ]:
name = "n2c"
base_path = os.path.abspath("../")
model_path = os.path.join(base_path, "models", name)

# Load translation model
model = None
model_name = 'n2c.h5'
model = load_model(os.path.join(model_path, model_name), custom_objects={'InstanceNormalization':InstanceNormalization})
model.summary()

In [ ]:
# Image paths
normal = os.path.join(base_path, "dataset/cxr/train/normal")
pneu = os.path.join(base_path, "dataset/cxr/train/pneumonia")

# Destinations
dest_path1 = os.path.join(base_path, "dataset/gen/from_normal")
dest_path2 = os.path.join(base_path, "dataset/gen/from_pneu")

helpers.create_directory(dest_path1)
helpers.create_directory(dest_path2)

In [ ]:
normal_path = [os.path.join(normal, x) for x in os.listdir(normal)]
pneu_path = [os.path.join(pneu, x) for x in os.listdir(pneu)]

len(normal_path), len(pneu_path)

In [ ]:
normal_path[:3]

In [ ]:
ct=1
for path in normal_path[:]:
    img = load_img(path)
    scipy.misc.imsave("{}/{}.jpg".format(dest_path1,ct), img)
    ct+=1

In [ ]:
name = "p2c"
base_path = os.path.abspath("../")
model_path = os.path.join(base_path, "models", name)

# Load translation model
model = None
model_name = 'p2c.h5'
model = load_model(os.path.join(model_path, model_name), custom_objects={'InstanceNormalization':InstanceNormalization})
model.summary()

In [ ]:
ct=1
for path in pneu_path[:]:
    img = load_img(path)
    scipy.misc.imsave("{}/{}_g2.jpg".format(dest_path2,ct), img)
    ct+=1